# ESPAAN

In [3]:
!rm -r spaan
!git clone git://github.com/nicolagulmini/spaan
!pip install Bio

Cloning into 'spaan'...
remote: Enumerating objects: 55, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 55 (delta 27), reused 28 (delta 12), pack-reused 0
Receiving objects: 100% (55/55), 406.20 KiB | 5.97 MiB/s, done.
Resolving deltas: 100% (27/27), done.
     |████████████████████████████████| 120 kB 5.1 MB/s 
     |████████████████████████████████| 2.3 MB 29.8 MB/s 


In [8]:
from spaan.utils import *

In [5]:
from Bio import SeqIO

adhesin_dataset = list(SeqIO.parse("spaan/adh.fasta", "fasta"))
not_adhesin_dataset = list(SeqIO.parse("spaan/not_adh.fasta", "fasta"))

test_protein = adhesin_dataset[0].seq

In [18]:
# features
feature_1 = aminoacids_frequencies(test_protein)
feature_2_1 = multiplet_frequencies(test_protein, 3) # error in multiplet frequencies
feature_2_2 = multiplet_frequencies(test_protein, 4)
feature_2_3 = multiplet_frequencies(test_protein, 5)
feature_3 = dipeptide_frequencies(test_protein)
feature_4 = charge_composition(test_protein)
feature_5 = hydrophobic_composition(test_protein)

(50,)
